## Lógica Computacional: 25/26
---
## TP1 - Ex2

$Grupo$ $05$ 

*   Vasco Ferreira Leite (A108399)
*   Gustavo da Silva Faria (A108575)
*   Afonso Henrique Cerqueira Leal (A108472)
---

Neste trabalho pretende-se generalizar o problema em várias direções:

- **Em primeiro lugar**, a grelha tem como parâmetro fundamental um inteiro que toma vários valores $n \in \{3,4,\dots\}$. Fundamentalmente a grelha passa de um quadrado com $n^{2} \times n^{2}$ células para um cubo tridimensional de dimensões $n^{2} \times n^{2} \times n^{2}$. Cada posição na grelha é representada por um triplo de inteiros $(i,j,k) \in \{1,\dots,n^{2}\}^{3}$.

- **Em segundo lugar**, as “regiões” que a definição menciona deixam de ser linhas, colunas e “sub-grids” para passar a ser qualquer “box” genérica com um número de células $\leq n^{3}$. Cada “box” é representada por um dicionário $D$ que associa, no estado inicial, cada posição $(i,j,k) \in D$ na “box” a um valor inteiro no intervalo $\{0,\dots,n^{3}\}$.

- Na inicialização da solução, as células associadas ao valor $0$ estão livres para ser instanciadas com qualquer valor não nulo. Se nessa fase, uma célula está associada a um valor não nulo, então esse valor está **fixo** e qualquer solução do problema não o modifica.

- A solução final do problema, tal como no problema original, verifica uma restrição do tipo *all-different* que, neste caso, tem a forma:  
  > Dentro de uma mesma “box”, todas as células têm valores distintos no intervalo  
  > $\{1,\dots,n^{3}\}$.

- Consideram-se neste problema duas formas básicas de “boxes”:
  - **“Cubos”** de $n^{3}$ células determinados pelo seu vértice superior, anterior, esquerdo.
  - **“Paths”** determinados pelo seu vértice de início, o vértice final e pela ordem entre os índices dos vértices sucessivos.

O **input** do problema é um conjunto de “boxes” e um conjunto de alocações de valores a células.


### Variáveis:

**Variáveis de decisão:**
- `cells[(i,j,k)]`: Valor da célula na posição $(i,j,k)$, domínio $[1, n^3]$

**Parâmetros:**
- `n`: Variável base do problema
- `size`: $n^2$ (dimensão de cada aresta do cubo)
- `domain_size`: $n^3$ tamanho do domínio (maior valor tomado por uma célula)
- `fixed_assignments`: Células com valores pré-definidos
- `cube_boxes`: Lista de cubos $n \times n \times n$
- `path_boxes`: Lista de paths entre vértices
- `generic_boxes`: Lista de boxes customizadas

In [1]:
from ortools.sat.python import cp_model

In [ ]:
def solve_sudoku_3d(n, fixed_assignments=None, cube_boxes=None, path_boxes=None, generic_boxes=None, time_limit=300):
    
    size = n * n
    domain_size = n * n * n
    
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit
    
    cells = {}
    for i in range(1, size + 1):
        for j in range(1, size + 1):
            for k in range(1, size + 1):
                cells[(i, j, k)] = model.NewIntVar(1, domain_size, f'cell_{i}_{j}_{k}')
    
    # Adicionar valores fixos
    if fixed_assignments:
        for (i, j, k), value in fixed_assignments.items():
            if value != 0:
                model.Add(cells[(i, j, k)] == value)
    
    # Adicionar constraints básicas
    _add_basic_constraints(model, cells, size)
    
    # Adicionar boxes cúbicas
    if cube_boxes:
        for start_i, start_j, start_k in cube_boxes:
            _add_cube_box(model, cells, n, size, start_i, start_j, start_k)
    
    # Adicionar boxes de path
    if path_boxes:
        for path_info in path_boxes:
            if len(path_info) == 2:
                start, end = path_info
                order_constraints = None
            else:
                start, end, order_constraints = path_info
            _add_path_box(model, cells, size, start, end, order_constraints)
    
    # Adicionar boxes genéricas
    if generic_boxes:
        for box_dict in generic_boxes:
            _add_generic_box(model, cells, box_dict)
    
    # Resolver o problema
    status = solver.Solve(model)
    
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        solution = {}
        for (i, j, k), var in cells.items():
            solution[(i, j, k)] = solver.Value(var)
        return solution
    else:
        print(f"Solução não encontrada. Status: {status}")
        return None

In [3]:
def _add_basic_constraints(model, cells, size):
    """Adiciona constraints básicas de linhas, colunas e pilhas"""
    
    # Restrições para linhas (i, j fixos, k varia)
    for i in range(1, size + 1):
        for j in range(1, size + 1):
            row_cells = [cells[(i, j, k)] for k in range(1, size + 1)]
            model.AddAllDifferent(row_cells)
    
    # Restrições para colunas (i, k fixos, j varia)
    for i in range(1, size + 1):
        for k in range(1, size + 1):
            col_cells = [cells[(i, j, k)] for j in range(1, size + 1)]
            model.AddAllDifferent(col_cells)
    
    # Restrições para pilhas (j, k fixos, i varia)
    for j in range(1, size + 1):
        for k in range(1, size + 1):
            stack_cells = [cells[(i, j, k)] for i in range(1, size + 1)]
            model.AddAllDifferent(stack_cells)

In [4]:
def _add_cube_box(model, cells, n, size, start_i, start_j, start_k):
    """Adiciona uma box do tipo cubo n×n×n"""
    cells_in_box = []
    
    for di in range(n):
        for dj in range(n):
            for dk in range(n):
                i = start_i + di
                j = start_j + dj  
                k = start_k + dk
                if 1 <= i <= size and 1 <= j <= size and 1 <= k <= size:
                    cells_in_box.append(cells[(i, j, k)])
    
    if len(cells_in_box) > 1:
        model.AddAllDifferent(cells_in_box)
    
    return cells_in_box

In [5]:
def _add_path_box(model, cells, size, start_vertex, end_vertex, order_constraints=None):
    """Adiciona uma box do tipo path entre dois vértices"""
    i1, j1, k1 = start_vertex
    i2, j2, k2 = end_vertex
    
    path_cells = []
    steps = max(abs(i2 - i1), abs(j2 - j1), abs(k2 - k1))
    
    # Interpolação linear entre vértices
    for t in range(steps + 1):
        if steps > 0:
            ratio = t / steps
            i = int(i1 + (i2 - i1) * ratio)
            j = int(j1 + (j2 - j1) * ratio)
            k = int(k1 + (k2 - k1) * ratio)
        else:
            i, j, k = i1, j1, k1
            
        if 1 <= i <= size and 1 <= j <= size and 1 <= k <= size:
            path_cells.append(cells[(i, j, k)])
    
    # Remove duplicatas mantendo a ordem
    unique_cells = []
    seen = set()
    for cell in path_cells:
        if cell not in seen:
            unique_cells.append(cell)
            seen.add(cell)
    
    if len(unique_cells) > 1:
        model.AddAllDifferent(unique_cells)
    
    # Aplica constraints de ordem se fornecidas
    if order_constraints:
        for (v1, v2) in order_constraints:
            model.Add(cells[v1] < cells[v2])
    
    return unique_cells

In [6]:
def _add_generic_box(model, cells, box_dict):
    """Adiciona uma box genérica definida por dicionário"""
    free_cells = []
    
    for (i, j, k), value in box_dict.items():
        if value == 0:  # Célula livre
            free_cells.append(cells[(i, j, k)])
        else:  # Célula com valor fixo
            model.Add(cells[(i, j, k)] == value)
    
    if len(free_cells) > 1:
        model.AddAllDifferent(free_cells)
    
    return free_cells

In [7]:
def print_solution_3d(solution, n, layer_by='k'):
    """
    Imprime a solução organizada por camadas
    
    Args:
        solution: dicionário com a solução
        n: parâmetro do problema
        layer_by: 'i', 'j', ou 'k' para organizar as camadas
    """
    if not solution:
        print("Nenhuma solução para imprimir")
        return
    
    size = n * n
    
    coord_ranges = {
        'i': range(1, size + 1),
        'j': range(1, size + 1), 
        'k': range(1, size + 1)
    }
    
    fixed_coord = layer_by
    moving_coords = [c for c in ['i', 'j', 'k'] if c != fixed_coord]
    
    for fixed_val in coord_ranges[fixed_coord]:
        print(f"\n--- Camada {fixed_coord} = {fixed_val} ---")
        
        # Cria matriz 2D para esta camada
        layer_matrix = []
        for mv1 in coord_ranges[moving_coords[0]]:
            row = []
            for mv2 in coord_ranges[moving_coords[1]]:
                coords = {fixed_coord: fixed_val, moving_coords[0]: mv1, moving_coords[1]: mv2}
                value = solution[(coords['i'], coords['j'], coords['k'])]
                row.append(value)
            layer_matrix.append(row)
        
        # Imprime a matriz
        for row in layer_matrix:
            print(' '.join(f'{val:3d}' for val in row))

In [8]:
# Configuração do problema para n=2
n = 5

# Valores fixos
fixed_assignments = {
    (1, 1, 1): 1,
    (4, 4, 4): 8,
    (2, 2, 2): 4
}

# Boxes cúbicas (cubos 2×2×2)
cube_boxes = [
    (1, 1, 1),  # Cubo começando em (1,1,1)
    (1, 3, 1),  # Cubo começando em (1,3,1)
]

# Boxes de path
path_boxes = [
    ((1, 1, 1), (4, 4, 4), None)  # Path diagonal sem constraints de ordem
]

# Boxes genéricas
generic_boxes = [
    {
        (1, 2, 1): 0,  # Livre
        (1, 2, 2): 3,  # Fixo
        (2, 2, 1): 0,  # Livre  
        (2, 2, 2): 0   # Livre
    }
]

print(f"Configuração: n={n}, dimensões {n*n}×{n*n}×{n*n}, domínio [1, {n*n*n}]")
print(f"Valores fixos: {len(fixed_assignments)}")
print(f"Boxes cúbicas: {len(cube_boxes)}")
print(f"Boxes de path: {len(path_boxes)}")
print(f"Boxes genéricas: {len(generic_boxes)}")

Configuração: n=5, dimensões 25×25×25, domínio [1, 125]
Valores fixos: 3
Boxes cúbicas: 2
Boxes de path: 1
Boxes genéricas: 1


In [9]:
# Resolver o problema
solution = solve_sudoku_3d(
    n=n,
    fixed_assignments=fixed_assignments,
    cube_boxes=cube_boxes,
    path_boxes=path_boxes,
    generic_boxes=generic_boxes,
    time_limit=300
)

if solution:
    print("\n✓ Solução encontrada!")
    print_solution_3d(solution, n, layer_by='k')
    
    # Mostrar alguns valores específicos
    print("\n--- Verificação de valores fixos ---")
    for coords in [(1,1,1), (2,2,2), (4,4,4)]:
        print(f"Célula {coords}: {solution[coords]}")
else:
    print("\n✗ Não foi possível encontrar solução")


✓ Solução encontrada!

--- Camada k = 1 ---
  1  10  13  18  26   2   3   4   5   6   7   8   9  11  12  14  15  16  17  19  20  21  22  23  24
 27  30  36  41  48  32  34   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 47  50  61  67  73  37  55   2   1   4   3   6   5   8   7  10   9  12  11  14  13  16  15  18  17
 78  81  86  94  96  77  83   3   4   1   2   7   8   5   6  11  12   9  10  15  16  13  14  19  20
 99 102 112 114 119  89 106   5   3   2   1   4   7   6   9   8  11  10  13  12  15  14  17  16  19
  2   1   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25
  3   2   1   5   4   7   6   9   8  11  10  13  12  15  14  17  16  19  18  21  22  20  24  25  23
  4   3   2   1   6   5   8   7  10   9  12  11  14  13  16  15  18  17  20  22  19  23  21  26  27
  5   4   6   2   1   3   9  10   7   8  13  14  11  12  17  18  19  15  16  23  24  25  20  21  22
  6   5   4   3   2   1  10  11  12   7   8   9  15  16